<a href="https://colab.research.google.com/github/cs229-mind/mind/blob/main/NRMS/NRMS_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NRMS: Neural News Recommendation with Multi-Head Self-Attention
The following documents are copied from MIND public document.

[NRMS](https://wuch15.github.io/paper/EMNLP2019-NRMS.pdf) is a neural news recommendation approach with multi-head selfattention. The core of NRMS is a news encoder and a user encoder. In the newsencoder, a multi-head self-attentions is used to learn news representations from news titles by modeling the interactions between words. In the user encoder, we learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. Besides, we apply additive
attention to learn more informative news and user 
representations by selecting important words and news.

## Properties of NRMS:
- NRMS is a content-based neural news recommendation approach.
- It uses multi-self attention to learn news representations by modeling the iteractions between words and learn user representations by capturing the relationship between user browsed news.
- NRMS uses additive attentions to learn informative news and user representations by selecting important words and news.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall and MINDlarge, please change the dowload source. Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to tranform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.




## Global settings and imports

In [1]:

import sys
#define util module path
path = '/home/rain/mind/NRMS'
sys.path.append(path)
data_path = '/home/rain/data'

import os
import numpy as np
import zipfile
from tqdm import tqdm
#import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from mind_model.dataloader import DataLoaderTrain, DataLoaderTest, test_iterator, train_iterator
from torch.utils.data import Dataset, DataLoader
from mind_model.preprocess import read_news, read_news_bert, get_doc_input, get_doc_input_bert
from mind_model.model_bert import ModelBert
from mind_model.parameters import parse_args
import torch.optim as optim
from pathlib import Path
import mind_model.utils as utils
import os
import sys
import csv
import logging
import datetime
from transformers import AutoTokenizer, AutoModel, AutoConfig, get_scheduler, AdamW
import torch
from torch import nn
from mind_model.nrms import TwoTowerModel as TwoTower

from mind_model.metrics import roc_auc_score, ndcg_score, mrr_score, ctr_score, cal_metric
import csv
import datetime
import time
import transformers

print("System version: {}".format(sys.version))
print("Transformers version: {}".format(transformers.__version__))
print("Tensorflow version: {}".format(tf.__version__))
print("Pytorch version: {}".format(torch.__version__))



System version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Transformers version: 4.12.5
Tensorflow version: 2.4.1
Pytorch version: 1.11.0.dev20211124+cu113


## Prepare parameters

In [2]:
def run_eval(dev_dataloader, model, run_eval_metric=True):
    eval_metrics = ['group_auc', 'mean_mrr', 'ndcg@5;10']
    group_impr_indexes = []
    group_labels = []
    group_preds = []
    start_time = time.time()

    tqdm_util_eval = tqdm(enumerate(dev_dataloader))

    with torch.no_grad():    
        for cnt, (impression_ids, user_ids, log_vecs, log_mask, news_vecs, news_bias, labels) in tqdm_util_eval:

            #count = cnt
            if args.enable_gpu:
                #user_ids = user_ids.cuda(non_blocking=True)
                log_vecs = log_vecs.cuda(non_blocking=True)
                log_mask = log_mask.cuda(non_blocking=True)
            user_vecs = model.user_encoder(log_vecs, log_mask).to(torch.device("cpu")).detach().numpy()

            for impression_id, user_vec, news_vec, bias, label in zip(
                    impression_ids, user_vecs, news_vecs, news_bias, labels):

                score = np.dot(
                    news_vec, user_vec
                )
                group_impr_indexes.append(impression_id)
                group_labels.append(label)
                group_preds.append(score)
    if run_eval_metric:
        print("Print final metrics")
        eval_res = cal_metric(labels=group_labels, preds=group_preds, metrics=eval_metrics)
        eval_info = ", ".join(
                    [
                        str(item[0]) + ":" + str(item[1])
                        for item in sorted(eval_res.items(), key=lambda x: x[0])
                    ]
                )

        logging.info(f"Time taken: {time.time() - start_time :.0f}s")
        print(f"Eval info: {eval_info} and time taken: {time.time() - start_time:.0f}s")
    else:
        print(f"Time taken: {time.time() - start_time}s")
    return group_impr_indexes, group_labels, group_preds

#group_impr_indexes, group_labels, group_preds = run_eval(dev_dataloader, model)

def model_fit(model, optimizer, train_dataloader, valid_dir, args, tokenizer, lr_scheduler=None,):
    logging.info('Training...')
    LOSS, ACC = [], []

    for ep in range(args.epochs):
        loss = 0.0
        accuracy = 0.0
        tqdm_util = tqdm(enumerate(train_dataloader))
        for cnt, (user_ids, log_ids, log_mask, input_ids, targets) in tqdm_util:

            if cnt > args.max_steps_per_epoch and args.subsampling:
                break

            if args.enable_gpu:
                log_ids = log_ids.cuda(non_blocking=True)
                log_mask = log_mask.cuda(non_blocking=True)
                input_ids = input_ids.cuda(non_blocking=True)
                #user_ids = user_ids.cuda(non_blocking=True)
                targets = targets.cuda(non_blocking=True)

            bz_loss, y_hat = model(input_ids, log_ids, log_mask, targets)
            # summary(model, [input_ids.shape, log_ids.shape, log_mask.shape, targets.shape], batch_size=16, device='cuda' if args.enable_gp else 'cpu')
            optimizer.zero_grad()
            bz_loss.backward()
            optimizer.step()
            if args.enable_lr_scheduler:
                lr_scheduler.step()

            loss += (bz_loss.data.float() - loss) / (cnt + 1)
            accuracy += (utils.acc(targets, y_hat) - accuracy) / (cnt + 1)
            if cnt % args.log_steps == 0:
                LOSS.append(loss.data)
                ACC.append(accuracy)
                #logging.info(
                tqdm_util.set_description(
                    '[{}] Ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(
                        hvd_rank, cnt * args.batch_size, loss.data,
                        accuracy))

        print('epoch: {} loss: {:.5f} accuracy {:.5f}'.format(ep + 1, loss, accuracy))
        
        dev_load = test_iterator(model=model, valid_dir=valid_dir, args=args, tokenizer=tokenizer)
        dev_dataloader = dev_load["iterator"]            
        _ = run_eval(dev_dataloader, model)
    return model



In [3]:
utils.setuplogger()
args = parse_args()
args.enable_hvd = False
args.batch_size = 32
args.news_dim = 400
args.word_emb_size = 768
args.user_attributes = None


train_dir = os.path.join(data_path, 'MINDsmall_train')
train_news_file = os.path.join(train_dir, r'news.tsv')
train_behaviors_file = os.path.join(train_dir, r'behaviors.tsv')

valid_dir = os.path.join(data_path, 'MINDsmall_dev')
valid_news_file = os.path.join(valid_dir, r'news.tsv')
valid_behaviors_file = os.path.join(valid_dir, r'behaviors.tsv')



tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
config = AutoConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)
bert_model = AutoModel.from_pretrained("bert-base-uncased",config=config)
bert_model = bert_model.cuda()

finetuneset={
'encoder.layer.10.attention.self.query.weight',
'encoder.layer.10.attention.self.query.bias',
'encoder.layer.10.attention.self.key.weight',
'encoder.layer.10.attention.self.key.bias',
'encoder.layer.10.attention.self.value.weight',
'encoder.layer.10.attention.self.value.bias',
'encoder.layer.10.attention.output.dense.weight',
'encoder.layer.10.attention.output.dense.bias',
'encoder.layer.10.attention.output.LayerNorm.weight',
'encoder.layer.10.attention.output.LayerNorm.bias',
'encoder.layer.10.intermediate.dense.weight',
'encoder.layer.10.intermediate.dense.bias',
'encoder.layer.10.output.dense.weight',
'encoder.layer.10.output.dense.bias',
'encoder.layer.10.output.LayerNorm.weight',
'encoder.layer.10.output.LayerNorm.bias',
'encoder.layer.11.attention.self.query.weight',
'encoder.layer.11.attention.self.query.bias',
'encoder.layer.11.attention.self.key.weight',
'encoder.layer.11.attention.self.key.bias',
'encoder.layer.11.attention.self.value.weight',
'encoder.layer.11.attention.self.value.bias',
'encoder.layer.11.attention.output.dense.weight',
'encoder.layer.11.attention.output.dense.bias',
'encoder.layer.11.attention.output.LayerNorm.weight',
'encoder.layer.11.attention.output.LayerNorm.bias',
'encoder.layer.11.intermediate.dense.weight',
'encoder.layer.11.intermediate.dense.bias',
'encoder.layer.11.output.dense.weight',
'encoder.layer.11.output.dense.bias',
'encoder.layer.11.output.LayerNorm.weight',
'encoder.layer.11.output.LayerNorm.bias',
'pooler.dense.weight',
'pooler.dense.bias',
'rel_pos_bias.weight',
'classifier.weight',
'classifier.bias'}
for name,param in bert_model.named_parameters():
    if name not in finetuneset:
        param.requires_grad = False

        


[INFO 2021-11-25 13:58:37,275] Namespace(mode='test', root_data_dir='~/mind/', dataset='MIND', train_dir='train', test_dir='test', filename_pat='behaviors*.tsv', model_dir='./model', batch_size=512, npratio=1, enable_gpu=True, enable_hvd=True, shuffle_buffer_size=10000, num_workers=2, filter_num=0, log_steps=100, epochs=4, lr=0.0001, news_attributes=['title'], process_uet=False, process_bing=False, num_words_title=24, num_words_abstract=50, num_words_body=50, num_words_uet=16, num_words_bing=26, user_log_length=50, word_embedding_dim=768, embedding_source='random', freeze_embedding=False, use_padded_news_embedding=False, padded_news_different_word_index=False, news_dim=64, news_query_vector_dim=200, user_query_vector_dim=200, num_attention_heads=20, user_log_mask=True, drop_rate=0.2, save_steps=500, max_steps_per_epoch=200000, load_ckpt_name='epoch-1-40000.pt', title_share_encoder=False, use_pretrain_news_encoder=False, pretrain_news_encoder_path='.', uet_agg_method='attention', model_

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Train Model

In [4]:

hvd_size, hvd_rank, hvd_local_rank = utils.init_hvd_cuda(
    args.enable_hvd, args.enable_gpu)

##Load Train data 
train_load = train_iterator(train_dir=train_dir, args=args, tokenizer=tokenizer)
category_dict = train_load["category_dict"]
domain_dict = train_load["domain_dict"]
subcategory_dict = train_load["subcategory_dict"]
train_dataloader = train_load["iterator"]


model = TwoTower(args, bert_model, len(category_dict), len(domain_dict), len(subcategory_dict))
args.optimizer = 'Adam'
args.enable_lr_scheduler = False
args.epochs=1
args.max_steps_per_epoch = 8000
args.subsampling = True

if args.enable_gpu:
    model = model.cuda()
    
if args.optimizer == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
elif args.optimizer == 'AdamW':
    optimizer = AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay, correct_bias=args.correct_bias)
else:
    optimizer = AdamW(model.parameters(), lr=args.lr)
    

if args.enable_lr_scheduler:
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=num_training_steps
    )

    
    

        


0it [00:00, ?it/s]/home/rain/anaconda3/envs/datascience/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
51282it [00:08, 5899.84it/s]
100%|██████████| 51282/51282 [00:00<00:00, 148898.14it/s]


In [5]:
## Initial fit
args.epochs=1
args.max_steps_per_epoch = 10
args.subsampling = True
model_fit(model=model, optimizer=optimizer, 
          train_dataloader=train_dataloader, 
          valid_dir=valid_dir, args=args, 
          tokenizer=tokenizer, lr_scheduler=None,)

[INFO 2021-11-25 13:59:10,783] Training...
[INFO 2021-11-25 13:59:10,784] DataLoader __iter__()
get_files: /home/rain/data/MINDsmall_train, behaviors*.tsv
files: ['/home/rain/data/MINDsmall_train/behaviors.tsv']
[INFO 2021-11-25 13:59:10,788] worker_rank:0, worker_size:1, shuffle:True, seed:0, directory:/home/rain/data/MINDsmall_train, files:['/home/rain/data/MINDsmall_train/behaviors.tsv']


0it [00:00, ?it/s]

[INFO 2021-11-25 13:59:10,807] data_paths: ['/home/rain/data/MINDsmall_train/behaviors.tsv']
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/home/rain/mind/NRMS/mind_model/dataloader.py:196: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  user_feature_batch = torch.LongTensor(user_feature_batch).cuda()
[0] Ed: 0, train_loss: 1.26315, acc: 0.62500: : 11it [00:15,  1.41s/it]


epoch: 1 loss: 1.50239 accuracy 0.52557


...

0it [00:00, ?it/s]

files: ['/home/rain/data/MINDsmall_dev/behaviors.tsv']
[INFO 2021-11-25 14:00:04,016] worker_rank:0, worker_size:1, shuffle:False, seed:0, directory:/home/rain/data/MINDsmall_dev, files:['/home/rain/data/MINDsmall_dev/behaviors.tsv']
[INFO 2021-11-25 14:00:04,017] visible_devices:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[INFO 2021-11-25 14:00:04,017] [StreamReader] path_len:1, paths: ['/home/rain/data/MINDsmall_dev/behaviors.tsv']


2286it [02:42, 14.06it/s]


Print final metrics
[INFO 2021-11-25 14:03:52,817] Time taken: 229s
Eval info: group_auc:0.4949, mean_mrr:0.2291, ndcg@10:0.2886, ndcg@5:0.2315 and time taken: 229s


TwoTowerModel(
  (news_encoder): NewsEncoder(
    (text_encoders): ModuleDict(
      (title): TextEmbedding(
        (bert_model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (dropout): Dropout(p=0.1, inplace=False)
     

In [6]:
## Continue fit
args.epochs=1
args.max_steps_per_epoch = 10
args.subsampling = True
model_fit(model=model, optimizer=optimizer, 
          train_dataloader=train_dataloader, 
          valid_dir=valid_dir, args=args, 
          tokenizer=tokenizer, lr_scheduler=None,)

[INFO 2021-11-25 14:03:52,954] Training...
[INFO 2021-11-25 14:03:52,955] DataLoader __iter__()
[INFO 2021-11-25 14:03:52,958] shut down pool.
get_files: /home/rain/data/MINDsmall_train, behaviors*.tsv


0it [00:00, ?it/s]

files: ['/home/rain/data/MINDsmall_train/behaviors.tsv']
[INFO 2021-11-25 14:03:52,963] worker_rank:0, worker_size:1, shuffle:True, seed:1, directory:/home/rain/data/MINDsmall_train, files:['/home/rain/data/MINDsmall_train/behaviors.tsv']
[INFO 2021-11-25 14:03:52,963] data_paths: ['/home/rain/data/MINDsmall_train/behaviors.tsv']


[0] Ed: 0, train_loss: 1.18901, acc: 0.50000: : 11it [00:15,  1.40s/it]


epoch: 1 loss: 0.93882 accuracy 0.55114


...

0it [00:00, ?it/s]

files: ['/home/rain/data/MINDsmall_dev/behaviors.tsv']
[INFO 2021-11-25 14:04:46,872] worker_rank:0, worker_size:1, shuffle:False, seed:0, directory:/home/rain/data/MINDsmall_dev, files:['/home/rain/data/MINDsmall_dev/behaviors.tsv']
[INFO 2021-11-25 14:04:46,873] visible_devices:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[INFO 2021-11-25 14:04:46,873] [StreamReader] path_len:1, paths: ['/home/rain/data/MINDsmall_dev/behaviors.tsv']


2286it [02:42, 14.09it/s]


Print final metrics
[INFO 2021-11-25 14:08:35,690] Time taken: 229s
Eval info: group_auc:0.4866, mean_mrr:0.2202, ndcg@10:0.2833, ndcg@5:0.2216 and time taken: 229s


TwoTowerModel(
  (news_encoder): NewsEncoder(
    (text_encoders): ModuleDict(
      (title): TextEmbedding(
        (bert_model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (dropout): Dropout(p=0.1, inplace=False)
     

## Save the model

In [7]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
    
model_path = os.path.join(data_path, "model_chkpt")
os.makedirs(model_path, exist_ok=True)
model_name = "bert"
series = 0
model_entity = os.path.join(model_path, f"model_chkpt{datetime.datetime.utcnow().strftime('%Y%m%d')}{model_name}")
#model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))
torch.save(model, model_entity)

Optimizer's state_dict:
state 	 {165: {'step': 22, 'exp_avg': tensor([[-7.7333e-04,  3.1400e-04, -6.0239e-04,  ...,  3.9728e-04,
         -1.9691e-04,  1.7475e-04],
        [-3.4753e-04, -4.9436e-05, -2.1236e-05,  ...,  2.2852e-05,
          1.0208e-04,  9.5246e-05],
        [ 1.9805e-04, -1.6407e-04, -8.8001e-05,  ...,  3.3140e-04,
          1.9215e-04,  2.1239e-04],
        ...,
        [ 2.1580e-04, -2.8088e-04,  2.1065e-04,  ..., -5.6990e-05,
          1.4667e-04,  2.1554e-05],
        [ 1.3241e-04, -2.5376e-05, -1.1091e-04,  ...,  7.3827e-05,
         -2.0303e-05,  6.4518e-05],
        [ 2.2005e-04, -1.3602e-04, -1.4250e-06,  ..., -1.8139e-05,
          1.0523e-05, -6.2108e-05]], device='cuda:0'), 'exp_avg_sq': tensor([[3.5999e-07, 4.9062e-07, 2.0715e-07,  ..., 1.2845e-07, 6.6545e-08,
         1.4495e-07],
        [4.9539e-08, 9.8816e-08, 3.0350e-08,  ..., 1.8435e-08, 1.7456e-08,
         4.3552e-08],
        [2.4605e-08, 5.5272e-08, 1.7329e-08,  ..., 3.3868e-08, 1.2324e-08,
     

In [8]:
test_model = torch.load(model_entity)


## Output Predcition File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [9]:
test_dir = os.path.join(data_path, 'MINDsmall_test')
test_news_file = os.path.join(test_dir, r'news.tsv')
test_behaviors_file = os.path.join(test_dir, r'behaviors.tsv')

test_load = test_iterator(model=model, valid_dir=test_dir, args=args, tokenizer=tokenizer)
test_dataloader = test_load["iterator"]

0it [00:00, ?it/s]/home/rain/anaconda3/envs/datascience/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
72023it [00:11, 6076.04it/s]
100%|██████████| 563/563 [00:50<00:00, 11.17it/s]


In [10]:
#group_impr_indexes, group_labels, group_preds = model.run_fast_eval(test_news_file, test_behaviors_file)

group_impr_indexes, group_labels, group_preds = run_eval(test_dataloader, model, run_eval_metric=False)

[INFO 2021-11-25 14:09:41,468] DataLoader __iter__()
get_files: /home/rain/data/MINDsmall_test, behaviors*.tsv


0it [00:00, ?it/s]

files: ['/home/rain/data/MINDsmall_test/behaviors.tsv']
[INFO 2021-11-25 14:09:41,473] worker_rank:0, worker_size:1, shuffle:False, seed:0, directory:/home/rain/data/MINDsmall_test, files:['/home/rain/data/MINDsmall_test/behaviors.tsv']
[INFO 2021-11-25 14:09:41,474] visible_devices:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[INFO 2021-11-25 14:09:41,475] [StreamReader] path_len:1, paths: ['/home/rain/data/MINDsmall_test/behaviors.tsv']


32it [00:02, 14.21it/s]

Time taken: 2.257753372192383s


In [11]:
group_impr_indexes1, group_labels1, group_preds1 = run_eval(test_dataloader, test_model, run_eval_metric=False)

[INFO 2021-11-25 14:09:43,732] DataLoader __iter__()
[INFO 2021-11-25 14:09:43,733] shut down pool.
get_files: /home/rain/data/MINDsmall_test, behaviors*.tsv


0it [00:00, ?it/s]

files: ['/home/rain/data/MINDsmall_test/behaviors.tsv']
[INFO 2021-11-25 14:09:43,737] worker_rank:0, worker_size:1, shuffle:False, seed:1, directory:/home/rain/data/MINDsmall_test, files:['/home/rain/data/MINDsmall_test/behaviors.tsv']
[INFO 2021-11-25 14:09:43,738] visible_devices:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[INFO 2021-11-25 14:09:43,738] [StreamReader] path_len:1, paths: ['/home/rain/data/MINDsmall_test/behaviors.tsv']


32it [00:02, 13.40it/s]

Time taken: 2.394899606704712s


In [26]:

predfile = f'prediction{series}.txt'
zip_file = f'prediction{series}.zip'
with open(os.path.join(data_path, predfile), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        #impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

1000it [00:00, 33231.69it/s]


In [28]:
#import zipfile
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, predfile), arcname='prediction.txt')
f.close()

## Reference
\[1\] Wu et al. "Neural News Recommendation with Multi-Head Self-Attention." in Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/